# 1. Import pacakage

In [1]:
# conda install -n sim_search ipykernel --update-deps --force-reinstall
import pandas as pd
import numpy as np

from utils.search import return_res
from utils.data_process import input_process, res_process
from utils.search import do_search
from utils.plot import plotly_plot
import matplotlib.pyplot as plt
import talib
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# 2. read data

In [6]:
df = pd.read_csv('./data/2330TW.csv')
df.head()
# df.rename({'Unnamed: 0': 'Time'}, axis=1, inplace=True)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2008-01-02,36.155183,36.155183,35.105517,35.280460,39300705,0.0,0.0
1,2008-01-03,34.930569,34.988883,34.638996,34.638996,35123067,0.0,0.0
2,2008-01-04,34.580682,35.222144,34.114164,34.988884,48851900,0.0,0.0
3,2008-01-07,33.239444,33.822592,32.539665,32.539665,104088436,0.0,0.0
4,2008-01-08,32.539672,33.006191,32.364728,32.656303,47704464,0.0,0.0


# 3. Input data proccess

In [7]:
s = 2497 # import your time here
e = 2518

In [8]:
input_proccess = input_process.full_wav_scalar(full_wav=np.array(df['Close']), start_p=s, end_p=e, scalar_mehod="min_max")
full_wav_std = input_proccess.apply_wav()
baseline_wav_std = full_wav_std[s:e]

# 4. Do algo search

In [ ]:
algo_li = ["shape_d", "mink_d", "corr_d", "dtw_d"] # 放定義好得相似度

res_d = {}

for alg in algo_li:
    search = do_search.search(baseline_wav=baseline_wav_std, full_wav=full_wav_std)
    res_d['{}'.format(alg)] = search.run_search_test(algo_option=alg)

# 5. Dictionay result intersection 

In [ ]:
# sort dict 
mink_d_sorted = dict(sorted(res_d['mink_d'].items(), key=lambda item: item[1])) # 小到大
corr_d_sorted = dict(sorted(res_d['corr_d'].items(), key=lambda item: item[1], reverse=True)) # 大到小
dtw_d_sorted = dict(sorted(res_d['dtw_d'].items(), key=lambda item: item[1]))
shape_d_sorted = dict(sorted(res_d['shape_d'].items(), key=lambda item: item[1]))

### 5.1 單一結果

In [10]:
x = return_res.get_sim_res(df, s, e, "corr_d")
print(x)

100%|██████████| 3570/3570 [00:00<00:00, 5990.04it/s]

['1155:1176']


In [47]:
clear_res_engine = res_process.clean_duplicate_li(list(corr_d_sorted.keys())[0:20])
print(clear_res_engine.ret_uq_range())


['1155:1176', '522:543', '1131:1152', '813:834', '2877:2898', '2730:2751', '2289:2310', '2658:2679', '1188:1209', '3378:3399', '2618:2639', '3042:3063', '3478:3499']


In [48]:
clear_res_engine = res_process.clean_duplicate_li(list(mink_d_sorted.keys())[0:20])
print(clear_res_engine.ret_uq_range())

['1131:1152', '522:543', '2289:2310', '1188:1209', '2877:2898', '813:834', '3194:3215', '1301:1322', '1512:1533', '2898:2919', '3478:3499', '2658:2679', '2730:2751', '3042:3063', '223:244']


In [49]:
clear_res_engine = res_process.clean_duplicate_li(list(dtw_d_sorted.keys())[0:50])
print(clear_res_engine.ret_uq_range())

['2730:2751', '2898:2919', '483:504', '2813:2834', '2451:2472', '1680:1701', '2372:2393', '1130:1151', '1187:1208', '355:376', '3041:3062', '2018:2039', '3480:3501', '1741:1762', '223:244', '2057:2078', '1303:1324', '523:544', '1095:1116', '1512:1533', '3194:3215', '714:735']


In [50]:
clear_res_engine = res_process.clean_duplicate_li(list(shape_d_sorted.keys())[0:10])
print(clear_res_engine.ret_uq_range())

['2239:2260', '1987:2008', '894:915', '763:784', '3473:3494', '622:643', '1892:1913', '1516:1537']


### 5.2 多結果合併

In [37]:
# get intersection
# priority dict放前面
res_process.res_d_set(d1 = corr_d_sorted, d2 = mink_d_sorted, rank_d1=5, rank_d2=5)

{'2329:2399', '2331:2401', '2820:2890', '3100:3170'}

In [38]:
print(s,e)

2330 2400


# Result Visualization

In [7]:
# s,e = 2330, 2380
f1 = plotly_plot.plot(df, s = s, e= e)
# print(s,e)

In [4]:
from dash import dcc

In [11]:
# 2138:2193 # 103:158 # 320:375 # 4422:4472 # 1212:1262 # 1150:1200 # 4940:4990 # 1100:1123 # 4598:4621 #2604:2627 #460:483 #4729:4752 # 3931:3954
f2 = plotly_plot.plot(df, s = 1155, e= 1176) #3547:3570 #19 42 # 38 61 #1551:1574 # 1942:1992 #3324:3374 # 1859:1909 # 1057:1107 # 1199:1269 # 1131:1152' # 2730:2751

In [12]:
def figures_to_html(figs, filename="dashboard.html"):   
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")

In [13]:
figures_to_html([f1,f2])